In [3]:
import cv2
import numpy as np
import tifffile
from skimage.registration import phase_cross_correlation
from skimage.exposure import rescale_intensity
from skimage.filters import gaussian
from skimage.util import img_as_uint
from skimage.transform import warp_polar, rotate
from skimage.feature import match_template

from skimage.filters import threshold_otsu
from tqdm import tqdm

from joblib import Parallel, delayed

import os

In [4]:
folder_to_process = ('01_', '03_', '05_', '07_', '09_', '11_')
in_dir = r'Y:\\coskun-lab\\Zhou\\4_HCR\\20210507_cytokines\\uc'
out_dir = r'Y:\\coskun-lab\\Zhou\\4_HCR\\20210507_cytokines\\uc\\00_registered_images\\v3'

In [5]:
def register_imgs(ims):
    # This function takes a list of image stacks, and takes the first image in the stack as a registration references
    
    for i in range(len(ims)-1,0,-1):
        
        # Find translational shift
        move = ims[i][:,:,0]
        station = ims[i-1][:,:,0]

        shift, _, _ = phase_cross_correlation(station, move)
        shift = shift.astype('int')

        if shift[0] < 0:
            for m in range(i,len(ims)):
                ims[m] = ims[m][-shift[0]:,:,:]
            for n in range(i):
                ims[n] = ims[n][:ims[i].shape[0],:,:]
        if shift[0] > 0:
            for n in range(i):
                ims[n] = ims[n][shift[0]:,:,:]
            for m in range(i,len(ims)):
                ims[m] = ims[m][:ims[i-1].shape[0],:,:]

        if shift[1] < 0:
            for m in range(i,len(ims)):
                ims[m] = ims[m][:,-shift[1]:,:]
            for n in range(i):
                ims[n] = ims[n][:,:ims[i].shape[1],:]
        if shift[1] > 0:
            for n in range(i):
                ims[n] = ims[n][:,shift[1]:,:]
            for m in range(i,len(ims)):
                ims[m] = ims[m][:,:ims[i-1].shape[1],:]
        
    # Find rotation shift between last cycle
    """
    station = ims[len(ims)-2][:,:,0]
    move = ims[len(ims)-1][:,:,0]
    
    radius = 750

    move_polar = warp_polar(move, radius = radius)
    station_polar = warp_polar(station, radius = radius)

    polar_shift,_,_ = phase_cross_correlation(move_polar, station_polar)
    angle = polar_shift[0]
    
    ims[len(ims)-1] = rotate(ims[len(ims)-1], angle=angle)
    """
    
    new_ims = Parallel(n_jobs=4)(delayed(img_as_uint)(i) for i in zip(ims))

    return new_ims

In [6]:
def reg_reference(ims):
    # This function takes a list of image stacks, and return a list of registration references
    
    new_ims = []
    for i in range(len(ims)):
        rescaled = rescale_intensity(ims[i])
        new_im = np.max(rescaled, axis=2)
        blurred = img_as_uint(gaussian(new_im))
        new_ims.append(rescale_intensity(blurred))
    return new_ims

In [7]:
gene_dict = {}
l = os.listdir(in_dir)
l.sort()
for n in l:
    temp = n.split('_')
    idx = np.nan
    try:
        idx = folder_to_process.index(temp[0]+'_')
    except ValueError:
        pass
    if idx == idx:
        cycle_name = 'cycle' + str(idx+1)
        gene_dict[cycle_name] = temp[1:]
gene_dict['cycle1'] = ['il8', 'il6', 'ccl11']
gene_dict['cycle3'] = ['empty'] + gene_dict['cycle3']
gene_dict['cycle4'] = ['empty'] + gene_dict['cycle4']
gene_dict['cycle5'] = ['empty'] + gene_dict['cycle5']
gene_dict['cycle6'] = ['ConA', 'PhA', 'WGA']
# gene_dict['cycle7'] = ['ConA', 'PhA', 'WGA']
gene_dict

{'cycle1': ['il8', 'il6', 'ccl11'],
 'cycle2': ['sox9', 'col1a1', 'actb'],
 'cycle3': ['empty', 'eef2', 'spp1'],
 'cycle4': ['empty', 'runx1', 'pdl1'],
 'cycle5': ['empty', 'gapdh', 'nanog'],
 'cycle6': ['ConA', 'PhA', 'WGA']}

In [62]:
rois = []
for n in l:
    if n.startswith('11_'):
        path = os.path.join(in_dir, n, 'processed_images')
        print(path)
        rois = os.listdir(path)
rois.sort()
home_dir = os.getcwd()
os.chdir(os.path.join(out_dir))
for roi in rois:
    os.mkdir(roi)
os.chdir(home_dir)

Y:\\coskun-lab\\Zhou\\4_HCR\\20210507_cytokines\\uc\11_segmentation\processed_images


In [63]:
# Run this block if previous block didn't get to os.chdir(home_dir)
os.chdir(home_dir)
os.getcwd()

'Y:\\coskun-lab\\Zhou\\4_HCR\\20210614_hchCulture\\00_software'

In [77]:
# Run this block if no segmentation is given
for i in tqdm(range(len(rois))):
    im_l = []
    roi = rois[i]
    for n in l:
        temp = n.split('_')
        roi_l = []
        if temp[0]+'_' in folder_to_process:
            path = os.path.join(in_dir, n, 'processed_images', roi)
            imn_l = os.listdir(path)
            imn_l.sort()
            for imn in imn_l:
                if imn.startswith('MAX'):
                    roi_l.append(tifffile.imread(os.path.join(path, imn)))
                if n.startswith('cycle1') and imn.startswith('cell'):
                    roi_l.append(tifffile.imread(os.path.join(path, imn)).astype('uint16'))
            if len(roi_l) == 0:
                continue
            roi_stack = np.dstack(tuple(roi_l))
            im_l.append(roi_stack)
        
    registered_im_l = register_imgs(im_l)
    
    keys = list(gene_dict.keys())
    name_l = []
    for j in range(len(keys)):
        key = keys[j]
        out_name = key+'_'+gene_dict[key][0]+'_'+gene_dict[key][1]+'_'+gene_dict[key][2]+'_'+roi+'.tif'
        name_l.append(os.path.join(out_dir, roi, out_name))
    Parallel(n_jobs=4)(delayed(tifffile.imsave)(j, i) for i,j in zip(registered_im_l, name_l))

100%|██████████████████████████████████████████████████████████████████████████████████| 90/90 [17:25<00:00, 11.62s/it]


In [64]:
# Run this block if segmentation is given along slide cycle 1 images
for i in tqdm(range(len(rois))):
    im_l = []
    roi = rois[i]
    for n in l:
        temp = n.split('_')
        roi_l = []
        if temp[0]+'_' in folder_to_process:
            path = os.path.join(in_dir, n, 'processed_images', roi)
            imn_l = os.listdir(path)
            imn_l.sort()
            for imn in imn_l:
                if imn.startswith('MAX'):
                    roi_l.append(tifffile.imread(os.path.join(path, imn)))
                if n.startswith('01') and imn.startswith('cell'):
                    roi_l.append(tifffile.imread(os.path.join(path, imn)).astype('uint16'))
            if len(roi_l) == 0:
                continue
            roi_stack = np.dstack(tuple(roi_l))
            im_l.append(roi_stack)

    registered_im_l = register_imgs(im_l)
    masks = []
    for i in range(4,registered_im_l[0].shape[3]):
        masks.append(registered_im_l[0][:,:,:,i])
    registered_im_l[0] = registered_im_l[0][:,:,:,:4]

    keys = list(gene_dict.keys())
    name_l = []
    for j in range(len(keys)):
        key = keys[j]
        out_name = key+'_'+gene_dict[key][0]+'_'+gene_dict[key][1]+'_'+gene_dict[key][2]+'_'+roi+'.tif'
        name_l.append(os.path.join(out_dir, roi, out_name))
    Parallel(n_jobs=4)(delayed(tifffile.imsave)(j, i) for i,j in zip(registered_im_l, name_l))
    mask_name_l = []
    for j in range(len(masks)):
        out_name = 'cell'+str(j+1) +'_Mask.tif'
        mask_name_l.append(os.path.join(out_dir, roi, out_name))
        masks[j] = masks[j].astype('uint8')
    Parallel(n_jobs=4)(delayed(tifffile.imsave)(j, i) for i,j in zip(masks, mask_name_l))

100%|██████████████████████████████████████████████████████████████████████████████████| 27/27 [04:50<00:00, 10.77s/it]


In [51]:
roi

'052'

In [8]:
# run this block if a ROI needs to be registered separately
im_l = []
# roi = rois[i]
roi = '006'
for n in l:
    temp = n.split('_')
    roi_l = []
    if temp[0]+'_' in folder_to_process:
        path = os.path.join(in_dir, n, 'processed_images', roi)
        imn_l = os.listdir(path)
        imn_l.sort()
        for imn in imn_l:
            if imn.startswith('MAX'):
                roi_l.append(tifffile.imread(os.path.join(path, imn)))
            if n.startswith('01') and imn.startswith('cell'):
                roi_l.append(tifffile.imread(os.path.join(path, imn)).astype('uint16'))
        if len(roi_l) == 0:
            continue
        roi_stack = np.dstack(tuple(roi_l))
        im_l.append(roi_stack)

registered_im_l = register_imgs(im_l)
masks = []
for i in range(4,registered_im_l[0].shape[3]):
    masks.append(registered_im_l[0][:,:,:,i])
registered_im_l[0] = registered_im_l[0][:,:,:,:4]

keys = list(gene_dict.keys())
name_l = []
for j in range(len(keys)):
    key = keys[j]
    out_name = key+'_'+gene_dict[key][0]+'_'+gene_dict[key][1]+'_'+gene_dict[key][2]+'_'+roi+'.tif'
    name_l.append(os.path.join(out_dir, roi, out_name))
Parallel(n_jobs=4)(delayed(tifffile.imsave)(j, i) for i,j in zip(registered_im_l, name_l))
mask_name_l = []
for j in range(len(masks)):
    out_name = 'cell'+str(j+1) +'_Mask.tif'
    mask_name_l.append(os.path.join(out_dir, roi, out_name))
    masks[j] = masks[j].astype('uint8')
Parallel(n_jobs=4)(delayed(tifffile.imsave)(j, i) for i,j in zip(masks, mask_name_l))

[None]